<a href="https://colab.research.google.com/github/Griotold/google_drive/blob/main/report/ch_5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차 검증과 그리드 서치
- 키워드 : 하이퍼 파라미터 (그리드서치 vs 랜덤서치)
- 데이터가 작을 때, 주로 사용
- 하이퍼파라미터
  + max_depth = 3, 정확도가 84%
  + 4는 안될까? 5는?
- 결론
  + 모르면 디폴트만 쓰자.
  + 가성비 (시간 대비 성능 보장 안됨)

## 검증 세트
- 테스트 세트로 일반화 성능을 올바르게 예측하려면 가능한 한 테스트 세트를 사용하지 말아야 한다.
- 테스트 세트는 1회성이다. 마지막에 딱 한 번 사용한다.
- 훈련 데이터를 훈련 데이터와 검증 데이터(validation set)로 재분할
- 훈련 세트에서 모델을 훈련하고 검증 세트로 모델을 평가한다.

## 현실
- 테스트 데이터가 별도로 존재하지 않는다.
- 전체 데이터 = 훈련 (6) : 검증 (2) : 테스트 (2)
  + 테스트 데이터는 모르는 데이터로 생각해야 한다.

## 데이터 불러오기

In [ ]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

# class 열을 타깃으로, 나머지 열을 특성 배열으로
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

## 훈련 세트와 테스트 세트로 분리

In [ ]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size = 0.2, random_state = 42)

## 훈련 세트를 다시 훈련 세트와 검증 세트로 분리
- sub_input과 sub_target이 진짜 훈련 세트
- val_input과 val_target이 검증 세트 
- 여기서도 test_size 매개변수를 0.2로 지정하여 train_input의 약 20%를 val_input으로 만든다.

In [ ]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size = 0.2, random_state = 42
)

- 단순히 train_test_split() 함수를 2번 적용해준 것 뿐이다.

In [ ]:
sub_input.shape, val_input.shape, test_input.shape

((4157, 3), (1040, 3), (1300, 3))

## 모델 만든 후 평가
- 훈련 세트로 훈련하고 검증 세트로 평가

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


- 모델이 과대적합되어 있음을 확인했다.
- 매개변수를 바꿔서 더 좋은 모데을 찾아야 한다.
- 그 전에 검증 세트에 관해 좀 더 알아야 할 것이 있다.

## 교차 검증
- 훈련 세트를 여러 폴드(겹)로 나눈 다음 한 폴드가 검증 세트 역할을 하고 나머지 폴드에서는 모델을 훈련한다. 교차 검증은 이런 식으로 모든 폴드에 대해 검증 점수를 얻어 평균하는 방법이다.
- 교차 검증의 목적 : 좋은 모델이 만들어 진다!
  + 좋은 모델 != 성능이 좋은 모델
  + 좋은 모델 = 과대적합이 아닌 모델
  + 좋은 모델 = 모형의 오차가 적은 모델
  + 좋은 모델 = 안정적인 모델
- 교재 245p
  + 모델평가 1 : 90% (소요시간 : 1시간)
  + 모델평가 2 : 85% (소요시간 : 1시간)
  + 모델평가 3 : 80% (소요시간 : 1시간)
- 단점 : 시간이 오래 걸린다.

## 교차 검증 함수
- cross_validate() 함수

In [ ]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)
# 이 함수는 fit_time, score_time, test_score 키를 가진 딕셔너리를 반환

{'fit_time': array([0.01153088, 0.01016831, 0.010638  , 0.01059246, 0.01636553]), 'score_time': array([0.00125289, 0.00107217, 0.00128126, 0.00225782, 0.00122762]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


- 처음 2개의 키는 각각 모델을 훈련하는 시간과 검증하는 시간을 의미한다.
- 각 키마다 5개의 숫자가 담겨 있다.
- 기본적으로 5-폴드(겹) 교차 검증을 수행한다. 

- 교차 검증의 최종 점수는 5개의 점수를 평균하여 얻는다.

In [ ]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


- 주의할 점은 cross_validate() 훈련 세트를 섞어 폴드를 나누지 않는다. 앞서 train_test_split()으로 전체 데이터를 섞어줬기 때문이다.
- 하지만 또 섞어주고 싶다면, 분할기(splitter)를 사용한다.
- 만약 훈련 세트를 섞은 후 10-폴드 교차 검증을 수행하려면 아래와 같이 코드를 작성한다.

In [ ]:
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝
- 내용만 알고가면 된다. 나중에 쓸일은 없다.
- 그리드 서치 vs 랜덤 서치
- 하이퍼파라미터를 꼭 하고 싶다면 랜덤 서치를 사용하자.
- 자동으로 잡아주는 라이브러리들이 등장하기 시작함.

### 그리드 서치
- 우리가 직접 값의 목록을 넣어줘야 한다.
- 쓰지 말자.

- 어떻게 사용하는지 간단하게 맛보자.
- 기본 매개변수를 사용한 결정 트리 모델에서 min_impurity_decrease 매개변수의 최적값을 찾아보자.
- 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 만든다.

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

# GridSearchCV 클래스에 탐색 대상 모델(DecisionTreeClassifier)과
# params 변수를 전달하여 그리드 서치 객체를 만든다.
# n_jobs = -1은 모든 CPU 코어를 사용한다는 의미. 기본값은 1이다.
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)

# 모델 훈련
# 이 메서드를 호출하면 그리드 서치 객체는 결정 트리 모델
# 에서 탐색할 매개변수를 바꿔가며 총 5번 실행한다. 
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

- GridSearchCV의 cv 매개변수 기본값은 5이다.
- 따라서 min_impurity_decrease 값마다 5-폴드 교차 검증을 수행한다.
- 결국 5 X 5 = 25개의 모델을 훈련하게 된다.

- 검증 점수가 가장 높은 모델의 매개변수 조합으로 알아서 다시 모델을 훈련한다.
- 이 모델은 gs 객체의 best_estimator_ 속성에 저장되어 있다.
- 그리드 서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장되어 있다.

In [ ]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))
print(dt)
print(gs.best_params_)

0.9615162593804117
DecisionTreeClassifier(min_impurity_decrease=0.0001, random_state=42)
{'min_impurity_decrease': 0.0001}


- 여기서는 0.0001이 가장 좋은 값으로 선택되었다.
- 각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_ 속성의 'mean_test_score'키에 저장되어 있다.

In [ ]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


- 첫 번째 값이 가장 큰 것 같다.
- 수동으로 고르는 것보다 넘파이 argmax() 함수를 사용하면 가장 큰 값의 인덱스를 뽑을 수 있다.

In [ ]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'max_depth': 29, 'min_impurity_decrease': 0.000437615171403628}


- 조금 더 복잡한 매개변수 조합을 탐색해보자.
- min_impurity_decrease는 노드를 분할하기 위한 불순도 감소 최소량을 지정한다.
- max_depth로 트리의 깊이를 지정한다.
- min_samples_split으로 노드를 나누기 위한 최소 샘플 수를 지정한다.

In [ ]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split' : range(2, 100, 10)}

- 이 매개 변수로 수행할 교차 검증 횟수는 
9 X 15 X 10 = 1350개이다.
- 기본 5-폴드 교차 검증을 수행하므로 만들어지는 모델의 수는 6,750개나 된다.
- 이제 그리드 서치를 실행해보자.

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

- 최상의 매개변수 조합을 확인하자.

In [ ]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


- 최상의 교차 검증 점수도 확인하자.

In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### 랜덤 서치
- 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있도록 확률 분포 객체를 전달한다.
- 싸이파이에서 2개의 확률 분포 클래스를 임포트한다.

### 싸이파이(scipy)
- 파이썬의 핵심 과학 라이브러리 중 하나로, 적분, 보간, 선형 대수, 확률 등을 포함한 수치 계산 전용 라이브러리이다.

In [ ]:
# stats(통계) 서브 패키지
# uniform : 균등분포
# randint : 임의의 정수
# 정리하면, 균등 분포에서 샘플링한다.
from scipy.stats import uniform, randint
rgen = randint(0, 10)
rgen.rvs(10)
# 정수값을 추출한다.

array([2, 2, 3, 2, 4, 5, 9, 5, 8, 4])

In [ ]:
np.unique(rgen.rvs(1000), return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 94,  76, 112,  91, 102, 108, 117,  95,  93, 112]))

- uniform 클래스의 사용법도 마찬가지.

In [ ]:
ugen = uniform(0, 1)
ugen.rvs(10)
# 실수값을 추출한다.

array([0.23763654, 0.30508564, 0.55129513, 0.67400913, 0.63637347,
       0.61471641, 0.8167904 , 0.99811336, 0.19127765, 0.04356988])

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# 탐색할 매개변수의 딕셔너리 만들기
# min_samples_leaf 매개변수 : 리프 노드가 되기 위한 최소 샘플의 개수
params = {
    'min_impurity_decrease' : uniform(0.0001, 0.001),
    'max_depth' : randint(20, 50),
    'min_samples_split' : randint(2, 25),
    'min_samples_leaf' : randint(1, 25),
}

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params,
                        n_iter = 100, n_jobs = -1, random_state = 42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f17909ce310>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f17909aa6d0>},
                   random_state=42)

- min_impurity_decrease는 0.0001에서 0.001 사이의 실수값을 샘플링한다.
- max_depth는 20에서 50 사이의 정수를 샘플링한다.
- min_samples_split은 2에서 25 사이의 정수를 샘플링.
- min_samples_leaf는 1에서 25사이의 정수를 샘플링.
- 샘플링 횟수는 사이킷런의 랜덤 서치 클래스인 RandomizedSearchCV의 n_iter 매개변수에 지정한다.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1,random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': range(5, 20),
                                        'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                                        'min_samples_split': range(2, 100, 10)},
                   random_state=42)

- 위 params에 정의된 매개변수 범위에서 총 100번을 샘플링하여 교차 검증을 수행하고 최적의 매개변수 조합을 찾게 된다.
- 앞서 그리드 서치보다 훨씬 교차 검증 수를 줄이면서 넓은 영역을 효과적으로 탐색하게 된 것이다.
- 최적의 매개변수 조합을 출력해보자.

In [ ]:
print(gs.best_params_)

{'min_samples_split': 12, 'min_impurity_decrease': 0.0005, 'max_depth': 11}


- 최고의 교차 검증 점수도 확인한다.

In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8681935292811135


- 그리드 서치와 마찬가지로 최적의 모델은 이미 전체 훈련세트로 훈련되어 best_estimator_ 속성에 저장되어 있다.
- 이 모델을 최종 모델로 결정하고 테스트 성능을 확인하자.

In [ ]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.8615384615384616
